In [130]:
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid
from torch_geometric.explain import Explainer, GraphMaskExplainer, CaptumExplainer, PGExplainer
from torch_geometric.nn import GATConv, GCNConv, TAGConv
from torch_geometric.transforms import NormalizeFeatures

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='../data/Planetoid', name='Cora', transform=NormalizeFeatures())
data = dataset[0].to(device)

In [152]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for _ in range(1, 201):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

explainer = Explainer(
    model=model,
    algorithm=CaptumExplainer('IntegratedGradients'),
    explanation_type='model',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
    node_mask_type='attributes',
    edge_mask_type='object',
    threshold_config=dict(
        threshold_type='topk',
        value=200,
    ),
)

node_index = 10
explanation = explainer(data.x, data.edge_index, index=node_index)
print(f'Generated explanations in {explanation.available_explanations}')

Generated explanations in ['node_mask', 'edge_mask']


In [132]:
explanation.visualize_graph("../img/graph_vis.png")

In [154]:
print(explanation.node_mask[torch.where(explanation.node_mask > 0)])

tensor([3.5553e-02, 3.5905e-02, 5.5290e-02, 5.5916e-03, 4.8894e-02, 7.9176e-02,
        1.0771e-01, 6.3364e-02, 4.6078e-02, 4.0422e-02, 1.3250e-01, 3.2805e-02,
        7.8347e-02, 3.2700e-02, 6.4930e-02, 8.3656e-04, 5.9388e-06, 2.2823e-05,
        7.0222e-04, 2.8074e-04, 2.5483e-04, 8.2423e-05, 1.4544e-03, 4.1283e-04,
        7.3639e-04, 7.7618e-04, 1.1929e-04, 7.0543e-03, 1.0056e-03, 3.4117e-03,
        2.8213e-03, 3.5388e-04, 1.7160e-02, 6.1518e-04, 2.3800e-03, 6.2024e-03,
        1.3757e-03, 1.3006e-02, 1.4562e-03, 2.9496e-02, 2.0776e-03, 1.8799e-02,
        3.4694e-02, 1.4978e-02, 3.1358e-04, 1.2365e-03, 1.0455e-03, 2.5666e-04,
        6.2331e-03, 1.2926e-04, 9.2028e-04, 2.3933e-03, 5.6735e-04, 4.6533e-03,
        5.0224e-04, 1.0011e-02, 7.6461e-04, 6.7683e-03, 1.2450e-02, 5.3428e-03,
        2.6490e-04, 3.8791e-04, 6.1683e-04, 9.3914e-05, 1.2243e-03, 5.0170e-04,
        4.9558e-04, 7.9004e-04, 2.9347e-03, 3.6781e-04, 4.1925e-03, 8.7446e-03,
        1.5617e-03, 1.6616e-03, 8.5977e-

In [146]:
class GAT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(dataset.num_features, 8, heads=8)
        self.conv2 = GATConv(64, dataset.num_classes, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


model = GAT().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(1, 201):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

explainer = Explainer(
    model=model,
    algorithm=CaptumExplainer('IntegratedGradients'),
    explanation_type='model',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
    node_mask_type='attributes',
    edge_mask_type='object',
    # threshold_config=dict(
    #     threshold_type='topk',
    #     value=10,
    # ),
)

node_index = torch.tensor([10])
explanation = explainer(data.x, data.edge_index, index=node_index)
print(f'Generated explanations in {explanation.available_explanations}')

Generated explanations in ['node_mask', 'edge_mask']


In [145]:
explanation.visualize_graph("../img/graph_vis.png")

In [163]:
print(explanation.node_mask[torch.where(explanation.node_mask > 0)])

tensor([3.5553e-02, 3.5905e-02, 5.5290e-02, 5.5916e-03, 4.8894e-02, 7.9176e-02,
        1.0771e-01, 6.3364e-02, 4.6078e-02, 4.0422e-02, 1.3250e-01, 3.2805e-02,
        7.8347e-02, 3.2700e-02, 6.4930e-02, 8.3656e-04, 5.9388e-06, 2.2823e-05,
        7.0222e-04, 2.8074e-04, 2.5483e-04, 8.2423e-05, 1.4544e-03, 4.1283e-04,
        7.3639e-04, 7.7618e-04, 1.1929e-04, 7.0543e-03, 1.0056e-03, 3.4117e-03,
        2.8213e-03, 3.5388e-04, 1.7160e-02, 6.1518e-04, 2.3800e-03, 6.2024e-03,
        1.3757e-03, 1.3006e-02, 1.4562e-03, 2.9496e-02, 2.0776e-03, 1.8799e-02,
        3.4694e-02, 1.4978e-02, 3.1358e-04, 1.2365e-03, 1.0455e-03, 2.5666e-04,
        6.2331e-03, 1.2926e-04, 9.2028e-04, 2.3933e-03, 5.6735e-04, 4.6533e-03,
        5.0224e-04, 1.0011e-02, 7.6461e-04, 6.7683e-03, 1.2450e-02, 5.3428e-03,
        2.6490e-04, 3.8791e-04, 6.1683e-04, 9.3914e-05, 1.2243e-03, 5.0170e-04,
        4.9558e-04, 7.9004e-04, 2.9347e-03, 3.6781e-04, 4.1925e-03, 8.7446e-03,
        1.5617e-03, 1.6616e-03, 8.5977e-